In [1]:
import csv
from datetime import datetime
from pysnmp.hlapi import *
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
target_ip = '192.168.11.100'
community = 'omegadejdar'
port = 161
oids = {
    'ISP_IN': '1.3.6.1.2.1.2.2.1.10.2',     # eth0 (WAN)
    'LAN_OUT': '1.3.6.1.2.1.2.2.1.16.2',    # eth0 
    'ISP_SPEED': '1.3.6.1.2.1.2.2.1.5.2'    # eth0 speed
}



In [3]:
def snmp_get(oid):
    iterator = getCmd(
        SnmpEngine(),
        CommunityData(community, mpModel=0),
        UdpTransportTarget((target_ip, port), timeout=3, retries=2),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )
    errorIndication, errorStatus, errorIndex, varBinds = next(iterator)
    if errorIndication or errorStatus:
        return None
    for varBind in varBinds:
        value = varBind[1]
        # Pokud je to číslo, vrať jako int
        if isinstance(value, (int, float)):
            return int(value)
        # Pokud je to řetězec, vrať jako string
        elif hasattr(value, 'prettyPrint'):
            return value.prettyPrint()
        else:
            return str(value)

In [4]:
import os
import csv
from datetime import datetime

# Bezpečný převod bajtů na MB
def safe_bytes_to_mb(value):
    try:
        return round(int(value) / (1024 * 1024), 2)
    except (TypeError, ValueError):
        return None

# Bezpečný převod bajtů/s na Mb/s
def safe_bytes_to_mbps(value):
    try:
        return round((int(value) * 8) / (1024 * 1024), 2)
    except (TypeError, ValueError):
        return None

# Získání dat
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
raw_isp_in = snmp_get(oids['ISP_IN'])
raw_lan_out = snmp_get(oids['LAN_OUT'])
raw_isp_speed = snmp_get(oids['ISP_SPEED'])

# Převod hodnot
isp_in_mb = safe_bytes_to_mb(raw_isp_in)
lan_out_mb = safe_bytes_to_mb(raw_lan_out)
isp_speed_mbps = safe_bytes_to_mbps(raw_isp_speed)

# Zápis do CSV s hlavičkou (i když soubor existuje, ale je prázdný)
file_path = 'traffic_log.csv'
write_header = not os.path.exists(file_path) or os.path.getsize(file_path) == 0

with open(file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if write_header:
        writer.writerow(['Time', 'ISP_IN_MB', 'LAN_OUT_MB', 'ISP_SPEED_Mbps'])
    writer.writerow([timestamp, isp_in_mb, lan_out_mb, isp_speed_mbps])


In [5]:
import pandas as pd
import os

# Načtení CSV souboru
file_path = 'traffic_log.csv'
df = pd.read_csv(file_path, names=['Time', 'ISP_IN_MB', 'LAN_OUT_MB', 'ISP_SPEED_Mbps'])

# Odstranění případných prázdných nebo chybných řádků
df = df[df['Time'] != 'Time']  # odstraní duplicitní hlavičku, pokud existuje


# Převedení sloupců na správné typy
df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
df['ISP_IN_MB'] = pd.to_numeric(df['ISP_IN_MB'], errors='coerce')
df['LAN_OUT_MB'] = pd.to_numeric(df['LAN_OUT_MB'], errors='coerce')
df['ISP_SPEED_Mbps'] = pd.to_numeric(df['ISP_SPEED_Mbps'], errors='coerce')

# Výpočet rozdílů
df['Delta_ISP_IN_MB'] = df['ISP_IN_MB'].diff().round(2)
df['Delta_LAN_OUT_MB'] = df['LAN_OUT_MB'].diff().round(2)
# df['Delta_ISP_SPEED_Mbps'] = df['ISP_SPEED_Mbps'].diff().round(2)
df['Delta_Time_hr'] = df['Time'].diff().dt.total_seconds().div(3600).round(2)


# Zobrazení výsledného DataFramu
print(df)

# Uložení pro další zpracování
output_path = 'traffic_log_dif.csv'
df.to_csv(output_path, index=False)



                  Time  ISP_IN_MB  LAN_OUT_MB  ISP_SPEED_Mbps  \
1  2025-10-05 13:11:00    3118.84        0.00          762.94   
2  2025-10-06 10:27:00    3763.39        0.00          762.94   
3  2025-10-06 11:50:14    3783.47        0.00          762.94   
4  2025-10-06 17:09:56    3819.19        0.00          762.94   
5  2025-10-06 17:40:04    3840.88      953.76          762.94   
6  2025-10-06 21:11:07    4075.29     3938.80          762.94   
7  2025-10-07 06:46:50     172.33     1141.27          762.94   
8  2025-10-08 06:48:59     815.99     3844.46          762.94   
9  2025-10-08 20:42:03    1190.74     3810.80          762.94   
10 2025-10-09 06:49:39    1607.51     1736.97          762.94   
11 2025-10-09 12:40:37    1767.57     1955.50          762.94   
12 2025-10-09 22:16:51    2229.73     2630.16          762.94   
13 2025-10-10 06:59:23    2388.89     3142.67          762.94   
14 2025-10-10 09:40:43    2443.36     3327.12          762.94   
15 2025-10-11 08:27:09   

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime

# Load data
file_path = '/Users/Marek/tableau_project/Rx_Tx_new/traffic_log_dif.csv'
df = pd.read_csv(file_path)

# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Filter rows from 7.10.2025 onwards and with positive values
start_date = datetime(2025, 10, 7)
filtered_df = df[(df['Time'] >= start_date) &
                 (df['Delta_LAN_OUT_MB'] > 0) &
                 (df['Delta_ISP_IN_MB'] > 0)]

# Plotting
plt.style.use('seaborn-v0_8')
fig, ax = plt.subplots(figsize=(12, 6))

x = filtered_df['Time']
lan = filtered_df['Delta_LAN_OUT_MB']
isp = filtered_df['Delta_ISP_IN_MB']

lan_sum = filtered_df["Delta_LAN_OUT_MB"].sum()
isp_sum = filtered_df["Delta_ISP_IN_MB"].sum()
total_sum = lan_sum + isp_sum

bar_width = 0.4
ax.bar(x - pd.Timedelta(hours=6), lan, width=bar_width, label='LAN OUT (MB)', color='steelblue', alpha=0.8)
ax.bar(x + pd.Timedelta(hours=6), isp, width=bar_width, label='ISP IN (MB)', color='orange', alpha=0.8)

# Fill under bars (approximate visual effect)
ax.fill_between(x, 0, lan, color='steelblue', alpha=0.2)
ax.fill_between(x, 0, isp, color='orange', alpha=0.2)

# Labels and title
ax.set_title('Spotřeba dat od 7.10.2025', fontsize=16)
ax.set_xlabel('Čas', fontsize=12)
ax.set_ylabel('Přenos dat (MB)', fontsize=12)
ax.legend()
plt.xticks(rotation=45)

# Add total summary text
summary_text = f"LAN = {lan_sum:.2f} MB\nISP = {isp_sum:.2f} MB\nCelkem = {total_sum:.2f} MB"
plt.text(0.99, 0.95, summary_text, transform=ax.transAxes,
         fontsize=12, verticalalignment='top', horizontalalignment='right',
         bbox=dict(facecolor='white', alpha=0.6, edgecolor='gray'))

# Save plot
output_path = '/Users/Marek/tableau_project/Rx_Tx_new/spotreba_dat_sloupcovy_graf.png'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.tight_layout()
plt.savefig(output_path)
plt.close()

print("Graf byl úspěšně vytvořen a uložen jako 'spotreba_dat_sloupcovy_graf.png'.")


Graf byl úspěšně vytvořen a uložen jako 'spotreba_dat_sloupcovy_graf.png'.


In [7]:
filtered_df

,Time,ISP_IN_MB,LAN_OUT_MB,ISP_SPEED_Mbps,Delta_ISP_IN_MB,Delta_LAN_OUT_MB,Delta_Time_hr
7,2025-10-08 06:48:59,815.99,3844.46,762.94,643.66,2703.19,24.04
10,2025-10-09 12:40:37,1767.57,1955.50,762.94,160.06,218.53,5.85
11,2025-10-09 22:16:51,2229.73,2630.16,762.94,462.16,674.66,9.60
12,2025-10-10 06:59:23,2388.89,3142.67,762.94,159.16,512.51,8.71
13,2025-10-10 09:40:43,2443.36,3327.12,762.94,54.47,184.45,2.69


In [8]:
!which python
!pip list | grep pysnmp


/Users/Marek/miniconda3/envs/snmp_fresh/bin/python
pysnmp                    4.4.12
